In [1]:
import pandas as pd
import numpy as np
import math as mt
from numpy import dot
from numpy.linalg import norm
from scipy import stats
from sklearn.decomposition import TruncatedSVD 

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sn
import IPython
import IPython.display

sn.set()
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

# 한글출력
matplotlib.rc('font', family='AppleGothic')  # MacOS
# matplotlib.rc('font', family='Malgun Gothic')  # Windows
plt.rcParams['axes.unicode_minus'] = False

In [2]:
def euclidean_distance(A, B):
    err = 0
    length = len(A)
    for idx in range(0, length):
        tmp = ((A[idx] - B[idx]) ** 2)
        err += tmp
        
    return mt.sqrt(err)

def cosine_similarity(A,B):
    return dot(A, B) / (norm(A) * norm(B))

def sumDiffer(A,B):
    length = len(A)
    err = 0
    for idx in range(0, length):
        err += ((A[idx] - B[idx]) ** 2)
    return mt.sqrt(err / length)

def improved_similarity(A, B, w):
    cos_sim = cosine_similarity(A, B)
    sum_diff = sumDiffer(A, B)
    w **= sum_diff
    
    return cos_sim * w

## an improved collaborative recommendation algorithm based on optimizer user similarity fit test
- 랜덤하게 (100, 32) 크기의 행렬 구조 생성 100 looping
- 어떠한 벡터를 정해서 해당 벡터에 대한 순위 매기기, sort_values
---
1. Euclidean Distance
    - 거리기반 유사도 측정값을 오름차순 정렬하고, 각 순위 사이의 방향기반(Cosine Similarity) 지수가 표준편차의 n% 를 넘어가면 잘못된 측정으로 Count
2. Cosine Similarity
    - 방향기반 유사도 측정값을 내림차순 정렬하고, 각 순위 사이의 거리기반(Euclidean Distance) 지수가 표준편차의 n% 를 넘어가면 잘못된 측정으로 Count
3. imp cos
    - 거리+방향 기반 유사도 측정값을 내림차순 정렬하고, 각 순위 사이의 방향기반과 거리기반 지수가 표준편차의 n%를 넘어가면 잘못된 측정으로 Count

In [27]:
def test_func(test_per):
    fit_cols = ['execute 거리기반, dis_err','execute 거리기반, cos_err', 'execute 거리기반, corr_err', 
                'execute 방향기반, dis_err', 'execute 방향기반, cos_err', 'execute 방향기반, corr_err',
                'execute 상관관계 기반, dis_err', 'execute 상관관계 기반, cos_err', 'execute 상관관계 기반, corr_err',
                'execute 거리+방향기반, dis_err', 'execute 거리+방향기반, cos_err', 'execute 거리+방향기반, corr_err']
    fit_df = pd.DataFrame(columns=fit_cols)

    for test_idx in range(1, 101):
        datas = np.round((np.random.rand(101,32) * 500))
        cols = ['유클리디안 거리', '코사인 유사도', '피어슨 상관계수','향상된 코사인 유사도']
        sim_df = pd.DataFrame(columns=cols)

        A = datas[0]
        B_values = datas[1:]
        imp_weight = 0.99

        for idx,B in enumerate(B_values):
            sim_df.loc[idx] = [
                euclidean_distance(A,B),
                cosine_similarity(A,B),
                stats.pearsonr(A,B)[0],
                improved_similarity(A, B, imp_weight)
            ]

        euc_std = sim_df['유클리디안 거리'].std()
        euc_std_per = euc_std * (test_per / 100)
        print("유클리디안 거리 표준편차 : {}\n".format(euc_std)
             +"유클리디안 거리 표준편차 ({}%) : {}\n".format(test_per,euc_std_per))

        cos_std = sim_df['코사인 유사도'].std()
        cos_std_per = cos_std * (test_per / 100)
        print("코사인 유사도 표준편차 : {}\n".format(cos_std)
             +"코사인 유사도 표준편차 ({}%) : {}\n".format(test_per,cos_std_per))
        
        corr_std = sim_df['피어슨 상관계수'].std()
        corr_std_per = corr_std * (test_per / 100)
        print("피어슨 상관계수 표준편차 : {}\n".format(corr_std)
             +"피어슨 상관계수 표준편차 ({}%) : {}\n".format(test_per,corr_std_per))


        sort_euc_based = sim_df.sort_values(by=['유클리디안 거리'])
        sort_cos_based = sim_df.sort_values(by=['코사인 유사도'], ascending=False)
        sort_corr_based = sim_df.sort_values(by=['피어슨 상관계수'], ascending=False)
        sort_imp_based = sim_df.sort_values(by=['향상된 코사인 유사도'], ascending=False)

        count_arr = []

        length = len(sort_euc_based)
        dis_count = 0
        cos_count = 0
        corr_count = 0
        for idx in range(1, length):
            dis_err = abs(sort_euc_based.iloc[idx - 1]['유클리디안 거리']
                     - sort_euc_based.iloc[idx]['유클리디안 거리']
                     )
            cos_err = abs(sort_euc_based.iloc[idx - 1]['코사인 유사도']
                     - sort_euc_based.iloc[idx]['코사인 유사도']
                     )
            corr_err = abs(sort_euc_based.iloc[idx - 1]['피어슨 상관계수']
                    - sort_euc_based.iloc[idx]['피어슨 상관계수'])

            if dis_err > euc_std_per:
                dis_count += 1
            if cos_err > cos_std_per:
                cos_count += 1
            if corr_err > corr_std_per:
                corr_count += 1

        print("표준편차를 넘어선 개수 : {}".format(cos_count))
        print("표준편차를 넘어선 개수 : {}\n".format(corr_count))
        count_arr.append(dis_count)
        count_arr.append(cos_count)
        count_arr.append(corr_count)

        length = len(sort_cos_based)
        dis_count = 0
        cos_count = 0
        corr_count = 0
        for idx in range(1, length):
            dis_err = abs(sort_cos_based.iloc[idx - 1]['유클리디안 거리']
                     - sort_cos_based.iloc[idx]['유클리디안 거리']
                     )
            cos_err = abs(sort_cos_based.iloc[idx - 1]['코사인 유사도']
                     - sort_cos_based.iloc[idx]['코사인 유사도']
                     )
            corr_err = abs(sort_cos_based.iloc[idx - 1]['피어슨 상관계수']
                     - sort_cos_based.iloc[idx]['피어슨 상관계수']
                     )

            if dis_err > euc_std_per:
                dis_count += 1
            if cos_err > cos_std_per:
                cos_count += 1
            if corr_err > corr_std_per:
                corr_count += 1

        print("표준편차를 넘어선 개수 : {}".format(dis_count))
        print("표준편차를 넘어선 개수 : {}\n".format(corr_count))
        count_arr.append(dis_count)
        count_arr.append(cos_count)
        count_arr.append(corr_count)
        
        length = len(sort_corr_based)
        dis_count = 0
        cos_count = 0
        corr_count = 0
        for idx in range(1, length):
            dis_err = abs(sort_corr_based.iloc[idx - 1]['유클리디안 거리']
                     - sort_corr_based.iloc[idx]['유클리디안 거리']
                     )
            cos_err = abs(sort_corr_based.iloc[idx - 1]['코사인 유사도']
                     - sort_corr_based.iloc[idx]['코사인 유사도']
                     )
            corr_err = abs(sort_corr_based.iloc[idx - 1]['피어슨 상관계수']
                     - sort_corr_based.iloc[idx]['피어슨 상관계수']
                     )

            if dis_err > euc_std_per:
                dis_count += 1
            if cos_err > cos_std_per:
                cos_count += 1
            if corr_err > corr_std_per:
                corr_count += 1

        print("표준편차를 넘어선 개수 : {}".format(dis_count))
        print("표준편차를 넘어선 개수 : {}\n".format(cos_count))
        count_arr.append(dis_count)
        count_arr.append(cos_count)
        count_arr.append(corr_count)
        
        length = len(sort_imp_based)
        dis_count = 0
        cos_count = 0
        corr_count = 0
        for idx in range(1, length):
            dis_err = abs(sort_imp_based.iloc[idx - 1]['유클리디안 거리']
                     - sort_imp_based.iloc[idx]['유클리디안 거리']
                     )
            cos_err = abs(sort_imp_based.iloc[idx - 1]['코사인 유사도']
                     - sort_imp_based.iloc[idx]['코사인 유사도']
                     )
            corr_err = abs(sort_imp_based.iloc[idx - 1]['피어슨 상관계수']
                     - sort_imp_based.iloc[idx]['피어슨 상관계수']
                     )

            if dis_err > euc_std_per:
                dis_count += 1
            if cos_err > cos_std_per:
                cos_count += 1
            if corr_err > corr_std_per:
                corr_count += 1
                
        count_arr.append(dis_count)                
        count_arr.append(cos_count)
        count_arr.append(corr_count)

        print("표준편차를 넘어선 개수 : {}".format(dis_count))
        print("표준편차를 넘어선 개수 : {}".format(cos_count))
        print("표준편차를 넘어선 개수 : {}\n".format(corr_count))

        fit_df.loc['test {}'.format(test_idx)] = count_arr

        IPython.display.clear_output()
        
    print("compare {}% 거리기반 mean error count, {}% 거리+방향 기반 mean error count".format(test_per, test_per))
    print("거리기반 =================> dis:{} cos:{} corr:{}".format(round(fit_df[fit_cols[0]].mean()),
                                                          round(fit_df[fit_cols[1]].mean()),
                                                         round(fit_df[fit_cols[2]].mean())))
    print("거리 + 방향 기반 ==========> dis:{} cos:{} corr:{}\n".format(round(fit_df[fit_cols[9]].mean()),
                                                                  round(fit_df[fit_cols[10]].mean()),
                                                          round(fit_df[fit_cols[11]].mean())))


    print("compare {}% 방향기반 mean error count, {}% 거리+방향 기반 mean error count".format(test_per, test_per))
    print("방향기반 =================> dis:{} cos:{} corr:{}".format(round(fit_df[fit_cols[3]].mean()),
                                                                 round(fit_df[fit_cols[4]].mean()),
                                                         round(fit_df[fit_cols[5]].mean())))
    print("거리 + 방향 기반 ==========> dis:{} cos:{} corr:{}\n".format(round(fit_df[fit_cols[9]].mean()),
                                                                  round(fit_df[fit_cols[10]].mean()),
                                                          round(fit_df[fit_cols[11]].mean())))
    
    print("compare {}% 상관계수 기반 mean error count, {}% 거리+방향 기반 mean error count".format(test_per, test_per))
    print("상관계수 기반 =================> dis:{} cos:{} corr:{}".format(round(fit_df[fit_cols[6]].mean()),
                                                            round(fit_df[fit_cols[7]].mean()),
                                                         round(fit_df[fit_cols[8]].mean())))
    print("거리 + 방향 기반 ==========> dis:{} cos:{} corr:{}\n".format(round(fit_df[fit_cols[9]].mean()),
                                                                  round(fit_df[fit_cols[10]].mean()),
                                                          round(fit_df[fit_cols[11]].mean())))

    fit_df

In [28]:
test_func(110)

compare 110% 거리기반 mean error count, 110% 거리+방향 기반 mean error count
거리기반 =================> dis:0 cos:7 corr:8
거리 + 방향 기반 ==========> dis:0 cos:2 corr:7

compare 110% 방향기반 mean error count, 110% 거리+방향 기반 mean error count
방향기반 =================> dis:6 cos:0 corr:11
거리 + 방향 기반 ==========> dis:0 cos:2 corr:7

compare 110% 상관계수 기반 mean error count, 110% 거리+방향 기반 mean error count
상관계수 기반 =================> dis:8 cos:11 corr:0
거리 + 방향 기반 ==========> dis:0 cos:2 corr:7



In [29]:
test_func(90)

compare 90% 거리기반 mean error count, 90% 거리+방향 기반 mean error count
거리기반 =================> dis:0 cos:11 corr:15
거리 + 방향 기반 ==========> dis:0 cos:5 corr:14

compare 90% 방향기반 mean error count, 90% 거리+방향 기반 mean error count
방향기반 =================> dis:10 cos:0 corr:19
거리 + 방향 기반 ==========> dis:0 cos:5 corr:14

compare 90% 상관계수 기반 mean error count, 90% 거리+방향 기반 mean error count
상관계수 기반 =================> dis:15 cos:19 corr:0
거리 + 방향 기반 ==========> dis:0 cos:5 corr:14



In [30]:
test_func(80)

compare 80% 거리기반 mean error count, 80% 거리+방향 기반 mean error count
거리기반 =================> dis:0 cos:16 corr:20
거리 + 방향 기반 ==========> dis:0 cos:8 corr:18

compare 80% 방향기반 mean error count, 80% 거리+방향 기반 mean error count
방향기반 =================> dis:16 cos:0 corr:22
거리 + 방향 기반 ==========> dis:0 cos:8 corr:18

compare 80% 상관계수 기반 mean error count, 80% 거리+방향 기반 mean error count
상관계수 기반 =================> dis:20 cos:23 corr:0
거리 + 방향 기반 ==========> dis:0 cos:8 corr:18



In [31]:
test_func(50)

compare 50% 거리기반 mean error count, 50% 거리+방향 기반 mean error count
거리기반 =================> dis:1 cos:35 corr:41
거리 + 방향 기반 ==========> dis:2 cos:25 corr:39

compare 50% 방향기반 mean error count, 50% 거리+방향 기반 mean error count
방향기반 =================> dis:36 cos:1 corr:44
거리 + 방향 기반 ==========> dis:2 cos:25 corr:39

compare 50% 상관계수 기반 mean error count, 50% 거리+방향 기반 mean error count
상관계수 기반 =================> dis:42 cos:45 corr:1
거리 + 방향 기반 ==========> dis:2 cos:25 corr:39



In [32]:
test_func(25)

compare 25% 거리기반 mean error count, 25% 거리+방향 기반 mean error count
거리기반 =================> dis:3 cos:58 corr:68
거리 + 방향 기반 ==========> dis:13 cos:49 corr:67

compare 25% 방향기반 mean error count, 25% 거리+방향 기반 mean error count
방향기반 =================> dis:59 cos:3 corr:71
거리 + 방향 기반 ==========> dis:13 cos:49 corr:67

compare 25% 상관계수 기반 mean error count, 25% 거리+방향 기반 mean error count
상관계수 기반 =================> dis:68 cos:70 corr:2
거리 + 방향 기반 ==========> dis:13 cos:49 corr:67



In [33]:
test_func(10)

compare 10% 거리기반 mean error count, 10% 거리+방향 기반 mean error count
거리기반 =================> dis:11 cos:82 corr:86
거리 + 방향 기반 ==========> dis:48 cos:77 corr:86

compare 10% 방향기반 mean error count, 10% 거리+방향 기반 mean error count
방향기반 =================> dis:82 cos:12 corr:87
거리 + 방향 기반 ==========> dis:48 cos:77 corr:86

compare 10% 상관계수 기반 mean error count, 10% 거리+방향 기반 mean error count
상관계수 기반 =================> dis:86 cos:87 corr:11
거리 + 방향 기반 ==========> dis:48 cos:77 corr:86



In [34]:
test_func(5)

compare 5% 거리기반 mean error count, 5% 거리+방향 기반 mean error count
거리기반 =================> dis:28 cos:90 corr:93
거리 + 방향 기반 ==========> dis:71 cos:88 corr:92

compare 5% 방향기반 mean error count, 5% 거리+방향 기반 mean error count
방향기반 =================> dis:90 cos:28 corr:93
거리 + 방향 기반 ==========> dis:71 cos:88 corr:92

compare 5% 상관계수 기반 mean error count, 5% 거리+방향 기반 mean error count
상관계수 기반 =================> dis:92 cos:93 corr:29
거리 + 방향 기반 ==========> dis:71 cos:88 corr:92

